# Predicting variation in EUR/USD exchange rate

In [1]:
#set the working directory
import os
os.chdir("/media/dftortosa/Windows/Users/dftor/Documents/diego_docs/industry/data_incubator/capstone_project/")

#check
os.getcwd()

'/media/dftortosa/Windows/Users/dftor/Documents/diego_docs/industry/data_incubator/capstone_project'

In [2]:
import pandas as pd
import numpy as np
#load the data
df_merged = pd.read_csv("results/sentiment_ecb_pricing.csv.gz", compression="gzip")
df_merged

,date,eur_usd_rate,previous_eur_pricing,percent_change_pricing,5_days_rolling_mean_pricing,10_days_rolling_mean_pricing,60_days_rolling_mean_pricing,120_days_rolling_mean_pricing,5_days_exp_rolling_mean_pricing,10_days_exp_rolling_mean_pricing,...,quantile_0.25_360_days_moving_neg_sent,quantile_0.35_360_days_moving_neg_sent,quantile_0.50_360_days_moving_neg_sent,quantile_0.65_360_days_moving_neg_sent,quantile_0.75_360_days_moving_neg_sent,quantile_0.9_360_days_moving_neg_sent,mean_360_days_moving_neg_sent,sum_360_days_moving_neg_sent,count_360_days_moving_neg_sent,std_360_days_moving_neg_sent
0,2008-10-28,1.2526,1.2460,0.529695,1.26470,1.30379,1.419718,1.492402,1.268561,1.295142,...,0.73375,0.7880,0.8525,0.89735,1.00,1.0,0.848519,1088.650,1420,0.130710
1,2008-11-04,1.2820,1.2822,-0.015598,1.28408,1.27439,1.400155,1.481216,1.281136,1.287815,...,0.73950,0.8020,0.8570,0.89400,1.00,1.0,0.848032,1105.834,1438,0.131298
2,2008-12-03,1.2623,1.2697,-0.582815,1.27110,1.27218,1.333905,1.431097,1.268733,1.270828,...,0.72800,0.7690,0.8420,0.89120,1.00,1.0,0.842147,1221.956,1569,0.133538
3,2008-12-10,1.2925,1.2838,0.677676,1.27804,1.27457,1.322672,1.419532,1.281809,1.277321,...,0.73500,0.7852,0.8510,0.90900,1.00,1.0,0.834483,1315.979,1693,0.135088
4,2008-12-11,1.3215,1.2925,2.243714,1.28994,1.27772,1.320527,1.417436,1.295039,1.285354,...,0.73500,0.7850,0.8500,0.90800,1.00,1.0,0.834156,1319.635,1698,0.134920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,2022-07-11,1.0098,1.0163,-0.639575,1.01816,1.03253,1.055547,1.084537,1.019291,1.028329,...,0.77000,0.8070,0.8540,0.90100,0.94,1.0,0.847649,112889.893,133180,0.118420
3431,2022-07-12,1.0042,1.0098,-0.554565,1.01320,1.02734,1.054278,1.083448,1.014261,1.023942,...,0.77000,0.8070,0.8540,0.90100,0.94,1.0,0.847631,113308.806,133677,0.118450
3432,2022-07-13,1.0067,1.0042,0.248954,1.01100,1.02284,1.053007,1.082417,1.011741,1.020807,...,0.77000,0.8060,0.8540,0.90100,0.94,1.0,0.847604,113669.587,134107,0.118466
3433,2022-07-14,1.0005,1.0067,-0.615874,1.00750,1.01902,1.051537,1.081365,1.007994,1.017115,...,0.76900,0.8060,0.8540,0.90100,0.94,1.0,0.847516,114110.464,134641,0.118483


In [3]:
#df_merged["percent_change_pricing"] = df_merged["eur_usd_rate"] - df_merged["previous_eur_pricing"]

Subset the dataframe with analysis data selecting only predictors

In [4]:
df_merged_modeling = df_merged.drop(["date", "eur_usd_rate", "percent_change_pricing", "previous_eur_pricing"], inplace=False, axis=1)
df_merged_modeling

,5_days_rolling_mean_pricing,10_days_rolling_mean_pricing,60_days_rolling_mean_pricing,120_days_rolling_mean_pricing,5_days_exp_rolling_mean_pricing,10_days_exp_rolling_mean_pricing,60_days_exp_rolling_mean_pricing,120_days_exp_rolling_mean_pricing,quantile_0.1_neg_sent,quantile_0.25_neg_sent,...,quantile_0.25_360_days_moving_neg_sent,quantile_0.35_360_days_moving_neg_sent,quantile_0.50_360_days_moving_neg_sent,quantile_0.65_360_days_moving_neg_sent,quantile_0.75_360_days_moving_neg_sent,quantile_0.9_360_days_moving_neg_sent,mean_360_days_moving_neg_sent,sum_360_days_moving_neg_sent,count_360_days_moving_neg_sent,std_360_days_moving_neg_sent
0,1.26470,1.30379,1.419718,1.492402,1.268561,1.295142,1.408270,1.454891,0.6476,0.67600,...,0.73375,0.7880,0.8525,0.89735,1.00,1.0,0.848519,1088.650,1420,0.130710
1,1.28408,1.27439,1.400155,1.481216,1.281136,1.287815,1.389191,1.441230,0.7776,0.80100,...,0.73950,0.8020,0.8570,0.89400,1.00,1.0,0.848032,1105.834,1438,0.131298
2,1.27110,1.27218,1.333905,1.431097,1.268733,1.270828,1.329658,1.390959,0.8288,0.89000,...,0.72800,0.7690,0.8420,0.89120,1.00,1.0,0.842147,1221.956,1569,0.133538
3,1.27804,1.27457,1.322672,1.419532,1.281809,1.277321,1.321813,1.381951,0.8425,0.85975,...,0.73500,0.7852,0.8510,0.90900,1.00,1.0,0.834483,1315.979,1693,0.135088
4,1.28994,1.27772,1.320527,1.417436,1.295039,1.285354,1.321803,1.380952,0.6099,0.68775,...,0.73500,0.7850,0.8500,0.90800,1.00,1.0,0.834156,1319.635,1698,0.134920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,1.01816,1.03253,1.055547,1.084537,1.019291,1.028329,1.057695,1.079113,0.6680,0.75100,...,0.77000,0.8070,0.8540,0.90100,0.94,1.0,0.847649,112889.893,133180,0.118420
3431,1.01320,1.02734,1.054278,1.083448,1.014261,1.023942,1.055941,1.077875,0.6935,0.77275,...,0.77000,0.8070,0.8540,0.90100,0.94,1.0,0.847631,113308.806,133677,0.118450
3432,1.01100,1.02284,1.053007,1.082417,1.011741,1.020807,1.054326,1.076699,0.6793,0.76600,...,0.77000,0.8060,0.8540,0.90100,0.94,1.0,0.847604,113669.587,134107,0.118466
3433,1.00750,1.01902,1.051537,1.081365,1.007994,1.017115,1.052562,1.075439,0.6790,0.76000,...,0.76900,0.8060,0.8540,0.90100,0.94,1.0,0.847516,114110.464,134641,0.118483


You can use the relative difference and other metrics of change of pricing.

In [14]:
y = df_merged["percent_change_pricing"].values
X = df_merged_modeling.values
print(y)
print(X)

[1.2526 1.282  1.2623 ... 1.0067 1.0005 1.0059]
[[1.26470000e+00 1.30379000e+00 1.41971833e+00 ... 1.08865000e+03
  1.42000000e+03 1.30709647e-01]
 [1.28408000e+00 1.27439000e+00 1.40015500e+00 ... 1.10583400e+03
  1.43800000e+03 1.31297684e-01]
 [1.27110000e+00 1.27218000e+00 1.33390500e+00 ... 1.22195600e+03
  1.56900000e+03 1.33538118e-01]
 ...
 [1.01100000e+00 1.02284000e+00 1.05300667e+00 ... 1.13669587e+05
  1.34107000e+05 1.18466049e-01]
 [1.00750000e+00 1.01902000e+00 1.05153667e+00 ... 1.14110464e+05
  1.34641000e+05 1.18482880e-01]
 [1.00542000e+00 1.01536000e+00 1.05027333e+00 ... 1.14468947e+05
  1.35070000e+05 1.18478718e-01]]


In [15]:
df_merged_modeling.columns

Index(['5_days_rolling_mean_pricing', '10_days_rolling_mean_pricing',
       '60_days_rolling_mean_pricing', '120_days_rolling_mean_pricing',
       '5_days_exp_rolling_mean_pricing', '10_days_exp_rolling_mean_pricing',
       '60_days_exp_rolling_mean_pricing', '120_days_exp_rolling_mean_pricing',
       'quantile_0.1_neg_sent', 'quantile_0.25_neg_sent',
       'quantile_0.35_neg_sent', 'quantile_0.50_neg_sent',
       'quantile_0.65_neg_sent', 'quantile_0.75_neg_sent',
       'quantile_0.9_neg_sent', 'mean_neg_sent', 'sum_neg_sent',
       'count_neg_sent', 'std_neg_sent',
       'quantile_0.1_10_days_moving_neg_sent',
       'quantile_0.25_10_days_moving_neg_sent',
       'quantile_0.35_10_days_moving_neg_sent',
       'quantile_0.50_10_days_moving_neg_sent',
       'quantile_0.65_10_days_moving_neg_sent',
       'quantile_0.75_10_days_moving_neg_sent',
       'quantile_0.9_10_days_moving_neg_sent', 'mean_10_days_moving_neg_sent',
       'sum_10_days_moving_neg_sent', 'count_10_days

In [16]:
predictors_null_model_array = df_merged_modeling.drop(df_merged_modeling.columns[["neg_sent" in column for column in df_merged_modeling.columns]], inplace=False, axis=1)
predictors_null_model_array

,5_days_rolling_mean_pricing,10_days_rolling_mean_pricing,60_days_rolling_mean_pricing,120_days_rolling_mean_pricing,5_days_exp_rolling_mean_pricing,10_days_exp_rolling_mean_pricing,60_days_exp_rolling_mean_pricing,120_days_exp_rolling_mean_pricing
0,1.26470,1.30379,1.419718,1.492402,1.268561,1.295142,1.408270,1.454891
1,1.28408,1.27439,1.400155,1.481216,1.281136,1.287815,1.389191,1.441230
2,1.27110,1.27218,1.333905,1.431097,1.268733,1.270828,1.329658,1.390959
3,1.27804,1.27457,1.322672,1.419532,1.281809,1.277321,1.321813,1.381951
4,1.28994,1.27772,1.320527,1.417436,1.295039,1.285354,1.321803,1.380952
...,...,...,...,...,...,...,...,...
3430,1.01816,1.03253,1.055547,1.084537,1.019291,1.028329,1.057695,1.079113
3431,1.01320,1.02734,1.054278,1.083448,1.014261,1.023942,1.055941,1.077875
3432,1.01100,1.02284,1.053007,1.082417,1.011741,1.020807,1.054326,1.076699
3433,1.00750,1.01902,1.051537,1.081365,1.007994,1.017115,1.052562,1.075439


In [17]:
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

null_model_raw = RandomForestRegressor()
full_model_raw = RandomForestRegressor()
    #https://www.nbshare.io/notebook/312837011/Decision-Tree-Regression-With-Hyper-Parameter-Tuning-In-Python/

In [18]:
#se parameters of the model
null_model_raw.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [19]:
from sklearn import metrics

#see the metrics that can be used in GridSearch
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [20]:
from sklearn.model_selection import cross_val_score, KFold
np.median(cross_val_score(RandomForestRegressor(), X, y, cv=KFold(n_splits=5, shuffle=True), scoring="r2", n_jobs=5))

0.9978683829616204

In [21]:
from sklearn.model_selection import cross_val_score, KFold
np.median(cross_val_score(RandomForestRegressor(), predictors_null_model_array, y, cv=KFold(n_splits=5, shuffle=True), scoring="r2", n_jobs=5))

0.9983319534149163

In [13]:
##use gridsearch to optimize the parameters of the model
#make imports
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ShuffleSplit

#make a dict with the parameters and values to be explored
    #https://www.nbshare.io/notebook/312837011/Decision-Tree-Regression-With-Hyper-Parameter-Tuning-In-Python/
    #https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
params={"max_depth" : [1,5,9,12, None]}

#shuffle
shuffle_split = ShuffleSplit(n_splits=5, test_size=0.25)
    #CHECK IN MORE DETAIL
    #This seems to be equivalent to: 
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        #But, it is usable for GridSearchCV
       #https://stackoverflow.com/questions/57672955/how-to-use-gridsearchcv-for-tuning-parameters-with-train-test-split-strategy

#prepare the gridsearch with the pipe across these parameter values using 3 cores
search_1 = GridSearchCV(null_model_raw, params, n_jobs=8, cv=shuffle_split, scoring="r2")
search_2 = GridSearchCV(full_model_raw, params, n_jobs=8, cv=shuffle_split, scoring="r2")
#search = RandomizedSearchCV(pipe_att, params, n_iter=20, cv=shuffle_split, verbose=100, random_state=42, n_jobs=-2, scoring="r2")
    #we can use random search if we have many parameters, so the total number of combinations of all parameters
        #will be veery high
        #https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
    #for scoring options
        #sklearn.metrics.SCORERS.keys())
        
#make the search for out data
search_1.fit(X=predictors_null_model_array, y=y)
search_2.fit(X=X, y=y)

#print the best score and parameters
print("Best parameter null model (CV score=%0.3f):" % search_1.best_score_)
print(search_1.best_params_)
print("Best parameter full model (CV score=%0.3f):" % search_2.best_score_)
print(search_2.best_params_)

KeyboardInterrupt: 

In [ ]:
search_2.cv_results_

In [ ]:
search_1.cv_results_

For now we are not using the GridSearch, we will explore this in the future

Let's see the coefficients (importance of each predictor)

In [ ]:
#print(ridge_model_1.feature_importances_)
#print(ridge_model_2.feature_importances_)

In [ ]:
null_model = RandomForestRegressor(random_state=565735)
full_model = RandomForestRegressor(random_state=565735)

null_model.fit(X=predictors_null_model_array, y=y)
full_model.fit(X=X, y=y)

In [ ]:
df_merged_modeling.columns

In [ ]:
from sklearn import metrics

null_model_pred = null_model.predict(X=predictors_null_model_array)
full_model_pred = full_model.predict(X=X)

In [ ]:
print("Null model: Mean absolute error:", metrics.mean_absolute_error(y, null_model_pred))
print("Null model: Mean squared error:", metrics.mean_squared_error(y, null_model_pred))
print("Null model: Root mean squared error:", metrics.mean_squared_error(y, null_model_pred, squared=False))
    #squared: If True returns MSE value, if False returns RMSE value.
print("Null model: R^2:", metrics.r2_score(y, null_model_pred))
    #0.83

In [ ]:
print("Full model: Mean absolute error:", metrics.mean_absolute_error(y, full_model_pred))
print("Full model: Mean squared error:", metrics.mean_squared_error(y, full_model_pred))
print("Full model: Root mean squared error:", metrics.mean_squared_error(y, full_model_pred, squared=False))
    #squared: If True returns MSE value, if False returns RMSE value.
print("Full model: R^2:", metrics.r2_score(y, full_model_pred))

Let's calculate the different in R2 between the two models in percentage:

In [ ]:
(metrics.r2_score(y, full_model_pred) - metrics.r2_score(y, null_model_pred))*100
    #1.79

In [ ]:
feat_importances = pd.Series(full_model.feature_importances_, index=df_merged_modeling.columns)
feat_importances.nlargest(5).plot(kind='barh')

In [ ]:
#open a plot with two panels
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))

#plot scatter against date and eur pricing
ax1.scatter(x=null_model_pred, y=y, s=0.5)
ax2.scatter(x=full_model_pred, y=y, s=0.5)

#set titles and axes labels
ax1.title.set_text("Null model")
ax1.set_ylabel("Observed % change")
ax1.set_xlabel("Predicted % change")
ax2.title.set_text("Full model (Twitter augmented)")
ax2.set_ylabel("Observed % change")
ax2.set_xlabel("Predicted % change")

#add R2
ax1.text(0.17, 0.8, 'R2 = ' + str(np.round(metrics.r2_score(y, null_model_pred), 4)), horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)
ax2.text(1.38, 0.8, 'R2 = ' + str(np.round(metrics.r2_score(y, full_model_pred), 4)), horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)

#close
plt.show()

### Next steps

- check moving averages
- Try to reach R2 of 0.9 modifying only twitter variables, so the model improving is only the twitter augumented
- Cross validation and parameter optimization
    - Compare RMSE between models in the evaluation datasets!!!
    - Avoid overfitting! You can sue many predictors, but use CV to avoid overftting!
- Include previous sentiment, this could be more useful for prediction? we do not need current sentiment...
    - Robert said to use the sentiment of the last week, month..
- Improve definition of the base model
- Try more models?
    - time series?
        - ARIMA considers shocks!
    - ensemble of random forest?
- Extend to other currencies?